In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# files_folder_tr = os.listdir('../input/hubmap-kidney-segmentation/train')
# files_folder_tr = sorted(files_folder_tr)
# files_folder_te = os.listdir('../input/hubmap-kidney-segmentation/test')
# files_folder_te = sorted(files_folder_te)

# filename_tr=[]
# filename_te=[]

# for file_name_tr in files_folder_tr: 
#     filename_tr.append(file_name_tr)
# for file_name_te in files_folder_te: 
#     filename_te.append(file_name_te)
    
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/hubmap-kidney-segmentation/sample_submission.csv
/kaggle/input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv
/kaggle/input/hubmap-kidney-segmentation/train.csv
/kaggle/input/hubmap-kidney-segmentation/test/aa05346ff.tiff
/kaggle/input/hubmap-kidney-segmentation/test/sample_submission.csv
/kaggle/input/hubmap-kidney-segmentation/test/3589adb90-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/2ec3f1bb9.tiff
/kaggle/input/hubmap-kidney-segmentation/test/3589adb90.tiff
/kaggle/input/hubmap-kidney-segmentation/test/d488c759a-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/57512b7f1-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/d488c759a.tiff
/kaggle/input/hubmap-kidney-segmentation/test/57512b7f1.tiff
/kaggle/input/hubmap-kidney-segmentation/test/aa05346ff-anatomical-structure.json
/kaggle/input/hubmap-kidney-segmentation/test/2ec3f1bb9-anatomical-structure.json
/kaggle/input/hubmap-kidney-

In [15]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)
from kaggle_datasets import KaggleDatasets
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
import os
import numpy as np 
import tensorflow as tf
import seaborn as sns 
import matplotlib.pyplot as plt
import json 
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.keras.utils.generic_utils import get_custom_objects
import os
import pandas as pd
import gc ### This is used to free up memory
import tifffile ## The input images are in .tiff format and can be parsed using this library


In [16]:
# to check runing on TPU 


print("Tensorflow version " + tf.__version__)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Tensorflow version 2.4.1
REPLICAS:  1


In [28]:

train_data = np.array(filename_tr[0:12])
validation_data = np.array(filename_tr[12:15])
train_data
validation_data

array(['2f6ecfcdf-anatomical-structure.json', '2f6ecfcdf.json',
       '2f6ecfcdf.tiff'], dtype='<U35')

In [18]:
with strategy.scope():
    def unet_model(OUTPUT_CHANNELS=1, tile_size=512, strides=1):
        initializer = 'he_normal'
        #keras.initializers.HeNormal()

        inputs = layers.Input(shape=[tile_size, tile_size, 3])

        ###LEVEL1
        d_conv1 = layers.Conv2D(filters=64, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(inputs)
        norm1 = layers.BatchNormalization()(d_conv1)                         
        d_conv2 = layers.Conv2D(filters=64, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(norm1)                             
        d_pool1 = layers.MaxPool2D(pool_size=2, strides=2, padding ='same')(d_conv2)
        ###LEVEL2
        d_conv3 = layers.Conv2D(filters=128, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_pool1)
        norm2 = layers.BatchNormalization()(d_conv3)                             
        d_conv4 = layers.Conv2D(filters=128, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(norm2)
        d_pool2 = layers.MaxPool2D(pool_size=2, strides=2, padding='same')(d_conv4)
        ##LEVEL3
        d_conv5 = layers.Conv2D(filters=256, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_pool2)
        d_conv6 = layers.Conv2D(filters=256, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_conv5)
        d_pool3 = layers.MaxPool2D(pool_size=2, strides=2, padding='same')(d_conv6)
        ###LEVEL4
        d_conv7 = layers.Conv2D(filters=512, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_pool3)
        d_conv8 = layers.Conv2D(filters=512, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_conv7)
        d_pool4 = layers.MaxPool2D(pool_size=2, strides=2, padding='same')(d_conv8)
        ###LEVEL5
        d_conv9 = layers.Conv2D(filters=1024, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_pool4)
        d_conv10 = layers.Conv2D(filters=1024, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(d_conv9)    

        ###Upsampling
        ###LEVEL1
        u_sample1 = layers.UpSampling2D(size=2)(d_conv10)
        u_conv11 = layers.Conv2D(filters=512, kernel_size=2, strides=strides, padding='same',
                                 kernel_initializer=initializer)(u_sample1)
        u_copy1 = layers.Concatenate(axis = 3)([d_conv8, u_conv11])

        u_conv12 = layers.Conv2D(filters=512, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_copy1)
        u_conv13 = layers.Conv2D(filters=512, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_conv12)
        ###Level2

        u_sample2 = layers.UpSampling2D(size=2)(u_conv13)
        u_conv14 = layers.Conv2D(filters=256, kernel_size=2, strides=strides, padding='same',
                                 kernel_initializer=initializer)(u_sample2)
        u_copy2 = layers.Concatenate(axis = 3)([d_conv6, u_conv14])
        u_conv15 = layers.Conv2D(filters=256, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_copy2)
        u_conv16 = layers.Conv2D(filters=256, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_conv15)


        ###Level3

        u_sample3 = layers.UpSampling2D(size=2)(u_conv16)
        u_conv17 = layers.Conv2D(filters=128, kernel_size=2, strides=strides, padding='same',
                                 kernel_initializer=initializer)(u_sample3)
        u_copy3 = layers.Concatenate(axis = 3)([d_conv4, u_conv17])
        u_conv18 = layers.Conv2D(filters=128, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_copy3)

        u_conv19 = layers.Conv2D(filters=128, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_conv18)

        ###Level4

        u_sample4 = layers.UpSampling2D(size=2)(u_conv19)
        u_conv20 = layers.Conv2D(filters=64, kernel_size=2, strides=strides, padding='same',
                                 kernel_initializer=initializer)(u_sample4)
        u_copy4 = layers.Concatenate(axis = 3)([d_conv2, u_conv20])
        u_conv21 = layers.Conv2D(filters=64, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_copy4)
        u_conv22 = layers.Conv2D(filters=64, kernel_size=3, strides=strides, padding='same',
                                 kernel_initializer=initializer, activation ='relu')(u_conv21)
        ###Last Level
        u_conv23 = layers.Conv2D(filters =1, kernel_size = OUTPUT_CHANNELS, activation='sigmoid')(u_conv22)

        return keras.Model(inputs=inputs, outputs=u_conv23)

In [19]:
def dice_coeff(y_true, y_pred):
        # add epsilon to avoid a divide by 0 error in case a slice has no pixels set
        # we only care about relative value, not absolute so this alteration doesn't matter
    _epsilon = 10 ** -7
    intersections = tf.reduce_sum(y_true * y_pred)
    unions = tf.reduce_sum(y_true + y_pred)
    dice_scores = (2.0 * intersections + _epsilon) / (unions + _epsilon)
    return dice_scores

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
  
tf.keras.utils.get_custom_objects().update({"dice": dice_loss})   

In [20]:
image_feature_description = {
      'image': tf.io.FixedLenFeature([], tf.string),
      'mask': tf.io.FixedLenFeature([], tf.string),
      'tile_No': tf.io.FixedLenFeature([], tf.int64),
      'image_id': tf.io.FixedLenFeature([], tf.string),
      'start_row_pixel': tf.io.FixedLenFeature([], tf.int64),
      'start_col_pixel': tf.io.FixedLenFeature([], tf.int64),
      'image_distribution0': tf.io.FixedLenFeature([], tf.string),
      'image_distribution1': tf.io.FixedLenFeature([], tf.string)
      
  }

def _parse_image_function2(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    
    image_id = single_example['image_id']
    start_row_pixel = single_example['start_row_pixel']
    start_col_pixel = single_example['start_col_pixel']
    
    num_channels = 3
    tile_size = 512
        
    
    image =  tf.io.decode_raw(single_example['image'],out_type='uint8')
   
    #img_array = tf.reshape( image, ( 1, tile_size, tile_size, num_channels))
    img_array = tf.reshape( image, (  tile_size, tile_size, num_channels))
    
    img_array = tf.cast(img_array, tf.float32) / 255.0
   
    mask =  tf.io.decode_raw(single_example['mask'],out_type='bool')
    
    mask = tf.reshape(mask, (tile_size,tile_size))
    
    mask = tf.cast(mask,tf.float32)
    
    image_distribution0 = tf.io.decode_raw(single_example['image_distribution0'], out_type = 'int64')
    image_distribution1 = tf.io.decode_raw(single_example['image_distribution1'], out_type = 'int64')
    
    image_distribution = (image_distribution0, image_distribution1)
    
    mtd = dict()
    mtd['img_index'] = single_example['image_id']
    mtd['tile_id'] = single_example['tile_No']
    mtd['start_col_pixel'] = single_example['start_col_pixel']
    mtd['start_row_pixel'] = single_example['start_row_pixel']
    struct = {
        'img_array': img_array,
        'mask': mask,
        'mtd': mtd,
        'image_distribution': image_distribution,
    } 
    return img_array, mask

In [21]:
def read_tf_dataset2(storage_file_path):
    encoded_image_dataset = tf.data.TFRecordDataset(storage_file_path, compression_type="GZIP")
    parsed_image_dataset = encoded_image_dataset.map(_parse_image_function2)
    return parsed_image_dataset

In [22]:
with strategy.scope():   
    model = unet_model()
    model.summary()
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=dice_loss,
              metrics=[dice_coeff])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       batch_normalization[0][0]        
______________________________________________________________________________________________

In [23]:
with strategy.scope():
    ACCELERATOR_TYPE = 'GPU'
    if ACCELERATOR_TYPE == 'GPU':
        batch_size = 128
        train_dataset = read_tf_dataset2(train_data) 
        train_dataset = train_dataset.batch(batch_size, drop_remainder=True).cache().prefetch(2)
        validation_dataset = read_tf_dataset2(validation_data)
        validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(2)
        steps_per_epoch = 100
        checkpointer = ModelCheckpoint('/kaggle/working/unet-gpu.h5', verbose=1)
        history = model.fit(train_dataset,batch_size=batch_size, epochs=30, callbacks=[checkpointer])
        model.save_weights("/kaggle/working/hubmap-gpu-unetf.h5")

history

2022-09-26 21:31:13.418816: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-26 21:31:13.422545: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000189999 Hz


Epoch 1/30


NotFoundError: 2 root error(s) found.
  (0) Not found:  0486052bb-anatomical-structure.json; No such file or directory
	 [[node IteratorGetNext (defined at tmp/ipykernel_18/1647595617.py:11) ]]
  (1) Not found:  0486052bb-anatomical-structure.json; No such file or directory
	 [[node IteratorGetNext (defined at tmp/ipykernel_18/1647595617.py:11) ]]
	 [[IteratorGetNext/_8]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_2858]

Function call stack:
train_function -> train_function
